In [6]:
# Standard imports
import copy
import corner
import numpy as np
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import display, Markdown, Math
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Astropy imports
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.units import Quantity
from astropy import constants as const

# SLSim imports
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
from slsim.lens import Lens
from slsim.LOS.los_pop import LOSPop

# Lenstronomy, HierArc imports
from lenstronomy.LensModel.lens_model import LensModel
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam
from likelihoods_parallel import beta2theta_e_ratio, beta_double_source_plane

# set global plotting parameters for academic paper
# plt.rcParams.update({
#     "font.size": 12,
#     "axes.labelsize": 14,
#     "axes.titlesize": 14,
#     "xtick.labelsize": 12,
#     "ytick.labelsize": 12,
#     "legend.fontsize": 12,
#     "figure.titlesize": 16,
#     "figure.figsize": (8, 6),
#     "lines.linewidth": 1.5,
#     "lines.markersize": 6,
#     "font.family": "serif",
#     "text.usetex": True,
# })

# Fit plane to the data
def fit_plane(x, y, z):
    """Fit a plane to the data points (x, y, z). The plane is z = ax + by + c."""

    # don't use nans or infs
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
    x = x[mask]
    y = y[mask]
    z = z[mask]

    A = np.c_[x, y, np.ones_like(x)]
    coeffs, _, _, _ = np.linalg.lstsq(A, z, rcond=None)
    return coeffs

# Find the scatter of the data points from the fitted plane
def find_scatter(x, y, z, coeffs, return_fit=False):
    """Find the scatter of the data points from the fitted plane. Return ``z - z_fit``."""

    # don't use nans or infs
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
    x = x[mask]
    y = y[mask]
    z = z[mask]

    z_fit = coeffs[0] * x + coeffs[1] * y + coeffs[2]
    scatter = z - z_fit
    if return_fit:
        return scatter, z_fit
    else:
        return scatter

In [5]:
# load the GGL data tables
sky_areas = ["30.0", "50.0", "100.0"]
GGL_data_tables = [Table.read(f"../data/GGL_{area}_SQDEG_RED_DEFLECTOR_BLUE_SOURCE.fits", format='fits') for area in sky_areas]
GGL_data_tables = {area: table for area, table in zip(sky_areas, GGL_data_tables)}
GGL_data_tables["30.0"]

lens_id,z_D,z_S,theta_E,sigma_v_D,mag_S_i,mag_S_r,mag_S_g,mag_S_z,mag_S_y,mag_D_i,mag_D_r,mag_D_g,mag_D_z,mag_D_y,size_D,e1_mass_D,e2_mass_D,e_mass_D,gamma_pl,R_e_kpc,Sigma_half_Msun/pc2,surf_bri_half_mag/arcsec2
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
0,0.6296670242460115,1.1757054120314414,0.930375658032011,291.0200969760373,22.182443197154036,22.54058232429268,23.147099654540135,21.590949058579774,21.33634728219201,19.506818141483798,20.61322276417026,21.985083010610126,19.11547936247149,18.82554987224743,0.7574645093782488,0.01997662989961611,0.12616603588082473,0.12773775617266725,2,5.176172538324463,6250.1643450566435,22.67463985073928
1,0.6027543477063477,3.352675366085025,0.5588299730446445,170.2979722047083,25.160044463259837,25.249580039397472,25.507132329489487,25.079995767365823,25.016860964620008,20.818753230197967,21.74220989864295,23.05752588807263,20.495282900725595,20.22515383600931,0.5246934980598951,0.1209737255562697,-0.03364347301588361,0.12556482609207942,2,3.5152752553179965,3397.044117022871,23.021188564646835
2,0.7910244037402018,2.9340606268735034,1.3128944804162774,284.9047775821409,24.66542881188305,24.935633019305403,25.175219552167423,24.498674357001175,24.44298888709739,20.51609562747607,21.502008138000463,22.21867323394173,19.92199845707534,19.656217240653284,1.1566069787585278,0.019146243411887837,0.0453503269003184,0.04922632209248423,2,8.650613627213724,4187.749439194667,23.833125579604776
3,0.4290200286461105,1.0692409242618046,0.6894558182309561,213.60644913070098,24.21476819988905,24.526247647047796,24.943336048849098,23.73563496518372,23.60257293623063,19.150865590266264,19.8013753848087,21.160933702401124,18.80773027354106,18.60024464414743,0.763044623490892,-0.16819176833885974,-0.02925137791797441,0.1707164726880591,2,4.275040610523187,3132.7131052084114,22.111999464332758
4,1.1168870942918339,3.8844990395848713,0.8586344671849839,245.2077297850457,25.415091657137722,25.523547922865056,26.223484687200454,25.25730770577308,25.170259245249,22.497920880812917,23.173366546972826,23.71492622820207,21.686448595234385,21.169799381839997,1.1072942286437617,-0.13569255618886952,0.051793782034361954,0.14524140478008504,2,9.075587124432014,3505.103307847054,25.29011893255856
5,0.49561942816276555,2.8724269066133163,1.035174661239692,226.57863249829794,25.844415987307528,26.011629084783266,26.160796651621283,25.723851791780504,25.66666339488716,19.37983430749174,20.14074852746731,21.52025111247319,19.007457054433416,18.754089538698878,0.5150564852655392,0.10723569466481823,0.010217524821601001,0.10772136289393186,2,3.129254563842385,5367.2628511691355,21.65870947288233
6,1.121106704319113,3.1062501070665114,0.5893584637468895,214.49747570917933,26.070373031016725,26.29875660358174,26.58313587951581,25.885526021108358,25.839537632545014,22.90636105627604,23.72171298633244,24.431860895752067,22.077989579126566,21.519786688581632,0.37293653700715157,-0.16669978330097554,-0.025078211805159498,0.16857560458125798,2,3.058746418283631,6825.197818378351,23.869301355714683
7,0.9750430757001669,3.8325718216180005,1.0781728933155974,263.0845195261197,26.861375596768397,26.917600019069233,27.52504874640433,26.721386607629576,26.64654018367385,21.834922056176154,22.677986939046384,23.44597096125105,20.9887236247057,20.563591336319988,0.7837697516353315,0.013481453023883849,-0.0726285524205078,0.07386918303550971,2,6.238653913041709,5644.739247284564,24.295211631341555
8,0.9980663192123288,2.363580490508556,1.1128414770523483,303.98887989858105,24.37348200867742,24.435955507120106,24.527020266365383,24.376473331285574,24.28168472791519,21.493828183318538,22.373400504843655,23.327023262523024,20.60296560177846,20.148740113489414,1.2310273070619713,-0.013023265729522792,-0.006570574835545709,0.01458690864889711,2,9.854420947584051,4260.4267314478475,25.072643487785864


In [ ]:
# fit the MFP for each dataset

coeffs_MFP_tables = {}
scatters_MFP_tables = {}
for sky_area, GGL_data_table in GGL_data_tables.items():
    # fit the MFP
    coeffs_MFP = fit_plane(
        np.log10(GGL_data_table["R_e_kpc"]),
        np.log10(GGL_data_table["Sigma_half_Msun/pc2"]),
        np.log10(GGL_data_table["sigma_v_D"])
    )
    coeffs_MFP_tables[sky_area] = coeffs_MFP
    # find the scatter
    scatter_MFP = find_scatter(
        np.log10(GGL_data_table["R_e_kpc"]),
        np.log10(GGL_data_table["Sigma_half_Msun/pc2"]),
        np.log10(GGL_data_table["sigma_v_D"]),
        coeffs_MFP,
        return_fit=False
    )
    scatters_MFP_tables[sky_area] = scatter_MFP

In [24]:
# make bins on the R_e vs Sigma_half plane and make pairings
# then compute the scatter in sigma_v_D from pairings
# and also the scatter in beta_E from pairings

num_bins = 20  # number of bins


# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)


# store pairing data template 
individual_pair_data = {
    'beta_E_pseudo': [],
    'beta_E_DSPL': [],
    'z_D1': [],
    'z_D2': [],
    'z_S1': [],
    'z_S2': [],
    'sigma_v_D1': [],
    'sigma_v_D2': [],
    'lens1_id': [],
    'lens2_id': [],
}

individual_pair_data_tables = {sky_area: copy.deepcopy(individual_pair_data) for sky_area in GGL_data_tables.keys()}

# loop over the sky areas
for sky_area, GGL_data_table in tqdm(GGL_data_tables.items(), desc="Processing Sky Areas", total=len(GGL_data_tables)):
    # make bins on the R_e vs Sigma_half plane
    R_e_bins = np.linspace(np.nanmin(GGL_data_table["R_e_kpc"]), np.nanmax(GGL_data_table["R_e_kpc"]), num_bins)
    Sigma_half_bins = np.linspace(np.nanmin(GGL_data_table["Sigma_half_Msun/pc2"]), np.nanmax(GGL_data_table["Sigma_half_Msun/pc2"]), num_bins)

    # make pairings
    for i in tqdm(range(len(R_e_bins) - 1)):
        for j in range(len(Sigma_half_bins) - 1):
            mask = (
                (GGL_data_table["R_e_kpc"] >= R_e_bins[i]) & (GGL_data_table["R_e_kpc"] < R_e_bins[i + 1]) &
                (GGL_data_table["Sigma_half_Msun/pc2"] >= Sigma_half_bins[j]) & (GGL_data_table["Sigma_half_Msun/pc2"] < Sigma_half_bins[j + 1])
            )
            if np.sum(mask) < 2:
                continue
            
            # get the data points in this bin
            data_points = GGL_data_table[mask]
            
            # make pairings
            for lens1, lens2 in combinations(data_points, 2):
                beta_E_pseudo = lens1['theta_E'] / lens2['theta_E']

                # check z_lens < z_source
                if lens1['z_D'] >= lens2['z_S'] or lens2['z_D'] >= lens1['z_S']:
                    continue

                beta_E_DSPL = beta_double_source_plane(
                    z_lens = np.mean([lens1['z_D'], lens2['z_D']]),
                    z_source_1= lens1['z_S'],
                    z_source_2= lens2['z_S'],
                    cosmo = cosmo_true,
                )
                beta_E_DSPL = beta2theta_e_ratio(
                    beta_dsp= beta_E_DSPL,
                    gamma_pl= 2,
                    lambda_mst= 1
                )

                individual_pair_data_tables[sky_area]['beta_E_pseudo'].append(beta_E_pseudo)
                individual_pair_data_tables[sky_area]['beta_E_DSPL'].append(beta_E_DSPL)
                individual_pair_data_tables[sky_area]['z_D1'].append(lens1['z_D'])
                individual_pair_data_tables[sky_area]['z_D2'].append(lens2['z_D'])
                individual_pair_data_tables[sky_area]['z_S1'].append(lens1['z_S'])
                individual_pair_data_tables[sky_area]['z_S2'].append(lens2['z_S'])
                individual_pair_data_tables[sky_area]['sigma_v_D1'].append(lens1['sigma_v_D'])
                individual_pair_data_tables[sky_area]['sigma_v_D2'].append(lens2['sigma_v_D'])
                individual_pair_data_tables[sky_area]['lens1_id'].append(lens1['lens_id'])
                individual_pair_data_tables[sky_area]['lens2_id'].append(lens2['lens_id'])

# convert to astropy tables
individual_pair_data_tables = {sky_area: Table(data=data) for sky_area, data in individual_pair_data_tables.items()}

Processing Sky Areas: 100%|██████████| 3/3 [01:34<00:00, 31.34s/it]


In [30]:
# print the scatter in sigma_v_D for pairs with |z_D1 - z_D2| < 0.01
for sky_area, individual_pair_data_table in individual_pair_data_tables.items():
    print(f"================= Sky Area: {sky_area} sq. deg. =================")
    mask = np.abs(individual_pair_data_table['z_D1'] - individual_pair_data_table['z_D2']) < 0.01
    print(f"Total number of pairs with |z_D1 - z_D2| < 0.01: {np.sum(mask)}")
    sigma_v_scatter = np.std(individual_pair_data_table[mask]['sigma_v_D1'] - individual_pair_data_table[mask]['sigma_v_D2']) / (np.mean(individual_pair_data_table[mask]['sigma_v_D1'] + individual_pair_data_table[mask]['sigma_v_D2']) / 2)
    print(f"Relative scatter in sigma_v_D for pairs with |z_D1 - z_D2| < 0.01: {sigma_v_scatter:.3f}")
    
    # scatter in beta_E
    beta_E_scatter = np.std(1 - individual_pair_data_table[mask]['beta_E_DSPL']/individual_pair_data_table[mask]['beta_E_pseudo'])
    print(f"Relative scatter in beta_E for pairs with |z_D1 - z_D2| < 0.01: {beta_E_scatter:.3f}")

================= Sky Area: 30.0 sq. deg. =================
Total number of pairs with |z_D1 - z_D2| < 0.01: 1925
Relative scatter in sigma_v_D for pairs with |z_D1 - z_D2| < 0.01: 0.088
Relative scatter in beta_E for pairs with |z_D1 - z_D2| < 0.01: 0.190
================= Sky Area: 50.0 sq. deg. =================
Total number of pairs with |z_D1 - z_D2| < 0.01: 7946
Relative scatter in sigma_v_D for pairs with |z_D1 - z_D2| < 0.01: 0.089
Relative scatter in beta_E for pairs with |z_D1 - z_D2| < 0.01: 0.194
================= Sky Area: 100.0 sq. deg. =================
Total number of pairs with |z_D1 - z_D2| < 0.01: 26537
Relative scatter in sigma_v_D for pairs with |z_D1 - z_D2| < 0.01: 0.095
Relative scatter in beta_E for pairs with |z_D1 - z_D2| < 0.01: 0.202


In [32]:
individual_pair_data_tables['100.0']

beta_E_pseudo,beta_E_DSPL,z_D1,z_D2,z_S1,z_S2,sigma_v_D1,sigma_v_D2,lens1_id,lens2_id
float64,float64,float64,float64,float64,float64,float64,float64,int64,int64
1.0110496148627404,1.0110496148627404,0.044607304251157574,0.044607304251157574,2.6655941332689594,1.5748354530601276,168.93096741142656,168.93096741142656,944,2631
1.201623044132491,1.0195876836443907,0.0949671934856237,0.22728288797427926,2.6350175849896766,2.002525570653582,160.1373009798414,155.66082221976995,239,272
1.267399956798713,0.9740490394124481,0.0949671934856237,0.3365285817102606,2.6350175849896766,3.867429002349289,160.1373009798414,152.47102085933417,239,566
1.0573378482351492,0.9911851405058196,0.0949671934856237,0.11000522762304844,2.6350175849896766,3.4543799764818086,160.1373009798414,155.8527843348213,239,805
1.1858527413154547,0.986861815300483,0.0949671934856237,0.38116142891838306,2.6350175849896766,3.074389936555832,160.1373009798414,162.04090791803304,239,906
1.3578289885693424,0.9891932711281063,0.0949671934856237,0.09523349381101759,2.6350175849896766,3.845971488669304,160.1373009798414,136.69402165296466,239,1017
0.9856042056590228,0.9960190699492028,0.0949671934856237,0.08004749031570269,2.6350175849896766,3.0143763955938563,160.1373009798414,160.13250349263362,239,1291
1.2435617151860021,0.9696102616089544,0.0949671934856237,0.25567117842352083,2.6350175849896766,4.962269353541303,160.1373009798414,148.98317633276082,239,1586
1.2603704110343177,1.0790625648983039,0.0949671934856237,0.2113679194074603,2.6350175849896766,1.1679982375303373,160.1373009798414,157.36464714119035,239,2018
